**`Webinar 2024-02-14`**

[YouTube Recording](https://www.youtube.com/watch?v=Kl2DBRobLU8&list=PLePKvgYhNOFxg3O7gXfmxaKp0EWXEhvNC&index=10)

# Book Ideation and Generation using Document Review and Long Form

### Overview

The goal is to write a quality educational book in the simplest way possible. We will go through the ideation phase to set the tone and topics of the book.

##### Project Flow

1. Create knowledge bases for each chapter we want in the book.
2. Get some user inputs in regard to overall theme and style.
3. For each chapter knowledge base, run a document review where the "final review" output is a chapter outline.
4. Feed each of the document review outlines into a long form generation call.

##### Initialization

##### `NOTE: To run this, you will need to replace with your own chapter KBs as well as book themes/guidance in the next 2 cells`

In [1]:
import superpowered
import requests
import time
import tqdm

from IPython.display import display, Markdown

SP_BASE_URL = 'https://api.superpowered.ai/v1'
SP_API_KEY_ID = ''
SP_API_KEY_SECRET = ''
API_AUTH = (SP_API_KEY_ID, SP_API_KEY_SECRET)

knowledge_base_ids = [
    '08844d05-4e09-479d-9770-6faebccfceb6',  # STOICISM
    'a0fd538c-06ec-43ae-849b-f9570caca2c6',  # EPICUREANISM
    '87401ada-dfcb-4e3a-afbc-8fe9ead62bb4',  # MINIMALISM
    '6fde0338-0856-477b-bfa0-439432dfbcd1',  # HEDONISM
    'a78019b4-c498-412b-9daf-737698fab4c5',  # EXISTENTIALISM
    '740b2668-ecab-4372-9bc4-6a01a4f88560',  # NIHILISM
    '183cee2f-2053-480b-8e86-9377489208c5',  # PRAGMATISM
    '0e75e305-b314-40a3-a4fb-7f357bda0c10',  # IDEALISM
]


##### User input describing goal for the book

In [2]:
BOOK_TITLE = 'Living a Good Life in the Age of AI and Automation'
BOOK_THEME = 'The general theme of this book is to look at what it means to live a good life in the context of stoicism, epicureanism, minimalism, hedonism, existentialism, nihilism, pragmatism, and idealism. Each chapter will cover one of these 8 philosophical schools of thought.'
BOOK_OVERALL_STYLE_GUIDANCE = 'The style should be widely accessible to the general public with a focus on how each of the philosophies are relevant in the modern age of AI and automation. It is important to use a variable vocabulary and make sure that nothing is repetitive and dry. Be fun and present ideas with examples. Avoid bland closing statements like "in summary", "in conclusion", etc. The book should be a fun and engaging read.'
BOOK_CHAPTER_STYLE_GUIDANCE = 'Each chapter should start with a general overview of the philosophy in question and explain how the concepts are relevant in the modern age of AI and automation. Give examples of what living by this philosophy would look like in scenarios like: "remote work and lack of social interacation", "complete automation with no jobs", "human population expantion", "virtual reality", "augmented reality", "cell phones", "immersive video games", etc.'

##### Helper functions that will run document reviews and long form plus stream results

- `run_and_display_review()` function will start a document review, show an active reading progress bar and then stream final review results as they happen.
- `submit_long_form_job()` just submits a long form job so we can generate all the chapters in parallel.


In [3]:
def run_and_display_review(kb: dict, ar_instructions: str, fr_instructions: str):
    url = f'{SP_BASE_URL}/knowledge_bases/{kb["id"]}/review'
    payload = {
        'active_reading_instructions': ar_instructions,
        'final_review_instructions': fr_instructions,
    }
    resp = requests.post(url, json=payload, auth=API_AUTH)
    final_review_display = display(Markdown('Review pending...'), display_id=True)
    progress_bar = tqdm.tqdm(total=1)
    progress_bar.set_description(f'Active Reading Progress for {kb["title"]}')

    # keep request job until it's completed (or failed)
    while resp.json()['status'] not in ['COMPLETE', 'FAILED']:
        resp = requests.get(resp.json()['status_url'], auth=API_AUTH)
        progress_bar.n = resp.json()['response']['active_reading_progress_pct']
        if progress_bar.n >= 1:
            final_review_display.update(Markdown(resp.json()["response"]["final_review"]))
        progress_bar.refresh()

    progress_bar.close()
    return resp.json()['response']['final_review']


def submit_long_form_job(kb: dict, prompt: str):
    url = f'{SP_BASE_URL}/long_form'
    payload = {
        'knowledge_base_ids': [kb['id']],
        'model': 'mistral-medium',
        'response_length': 'long',
        'prompt': prompt,
    }
    resp = requests.post(url, json=payload, auth=API_AUTH)
    return resp.json()['id']


##### Loop through each knowledge base and run the document review

In [4]:
active_reading_instructions = f"""\
You are reading through a knowledge base on {{philosophy}}. Please extract information that is relevant to living a good life.

The infomration that is extracted will be used to write a chapter about {{philosophy}} for a book with the following theme: {BOOK_THEME}

Your output should be a list of key points and examples that could be relevant to the modern age of AI and automation. The information you are reading through might not directly mention AI and automation so it's up to you to decide if the information is relevant or not.
"""

final_review_instructions = f"""\
Given the list key points and examples from the active reading, please generate a chapter outline. The book must follow these style guidelines:

1. {BOOK_OVERALL_STYLE_GUIDANCE}
2. {BOOK_CHAPTER_STYLE_GUIDANCE}

The chapter outline should contain a list of detailed section descriptions. Please try to keep the number of sections between 5 and 7
"""

outlines = []
for kb_id in knowledge_base_ids:
    # get the knowledge base object
    kb_url = f'{SP_BASE_URL}/knowledge_bases/{kb_id}'
    resp = requests.get(kb_url, auth=API_AUTH)
    kb = resp.json()

    outline = run_and_display_review(kb, active_reading_instructions.format(philosophy=kb['title']), final_review_instructions)
    outlines.append((kb, outline))

Chapter Outline for STOICISM [CH. 1]

1. Introduction to Stoicism and Its Modern Relevance
   - Brief history of Stoicism, from Zeno of Citium to its influence on Roman figures like Seneca and Marcus Aurelius.
   - Explanation of Stoicism's core belief in virtue as the path to a good life.
   - Discussion on the resurgence of Stoicism in the context of AI and automation, and its implications for modern ethical behavior and mental resilience.

2. The Stoic Virtues in an Automated World
   - Exploration of the four cardinal virtues: wisdom, courage, temperance, and justice.
   - Application of these virtues to scenarios such as job automation, remote work, and the ethics of AI decision-making.
   - Examples of how Stoic virtues can guide personal development and societal norms in an increasingly digital and virtual landscape.

3. Stoicism and the Dichotomy of Control
   - Explanation of the Stoic principle of focusing on what is within our control and accepting what is not.
   - Analysis of how this principle can be applied to the unpredictability of technological advancements and the changing job market.
   - Real-life examples of individuals and organizations embodying this principle in the face of rapid change.

4. Emotions and Rationality in the Age of Information Overload
   - Discussion of the Stoic approach to emotions, distinguishing between healthy and unhealthy passions.
   - Insights into how Stoicism can help navigate the emotional challenges of social media, information overload, and virtual interactions.
   - Case studies of Stoic-inspired practices in modern psychotherapy and cognitive-behavioral approaches.

5. Stoicism and Social Roles in a Connected World
   - Examination of the Stoic concept of embracing one's social role and the idea of cosmopolitanism.
   - Consideration of how Stoicism can inform our approach to global citizenship, online communities, and environmental responsibility.
   - Stories of individuals and communities practicing Stoic principles to foster cooperation and empathy in a diverse world.

6. The Stoic Path to Contentment Amidst Technological Disruption
   - Analysis of how Stoicism teaches contentment and equanimity, even in the face of significant life changes due to technology.
   - Discussion on the role of Stoicism in finding personal fulfillment when traditional paths to success are altered by AI and automation.
   - Profiles of people who have found balance and purpose by applying Stoic wisdom to their lives in the digital era.

7. Conclusion: Stoicism as a Guide for the Future
   - Recap of the key Stoic principles and their relevance to contemporary challenges.
   - Reflection on the enduring wisdom of Stoicism and its potential to shape a future where technology and human values coexist harmoniously.
   - Encouragement for readers to explore Stoic practices in their own lives as a means to navigate the complexities of the modern world.

Active Reading Progress for STOICISM [CH. 1]: 100%|██████████| 1/1 [01:18<00:00, 78.69s/it]       


Chapter Outline for "Epicureanism [CH. 2]"

1. Introduction to Epicureanism: The Pursuit of Pleasure in the Age of AI
   - Overview of Epicureanism and its foundation by Epicurus in 307 BCE.
   - Explanation of the Epicurean pursuit of pleasure, defined as the absence of pain and fear.
   - Relevance of Epicurean philosophy in the context of modern technology, AI, and automation.

2. The Epicurean Lifestyle: Simplicity and Tranquility Amidst Technological Complexity
   - Discussion on the Epicurean advocacy for a simple life and the withdrawal from politics.
   - Application of Epicurean simplicity in the era of remote work, social media, and the digital rat race.
   - Examples of how to maintain tranquility and mental well-being in a world dominated by technology.

3. Sensory Perception and Knowledge: Epicurean Empiricism in a Virtual World
   - Exploration of Epicurean empirical epistemology and its reliance on the senses.
   - Contrast with the virtual and augmented realities that challenge traditional sensory experiences.
   - Strategies for discerning truth and maintaining mental clarity in an age of information overload.

4. Desires and Moderation: Navigating Consumerism and Digital Temptations
   - Examination of the Epicurean classification of desires and the emphasis on controlling them.
   - Reflection on the impact of consumerism and digital temptations on our desires and how to manage them.
   - Insights into achieving aponia and ataraxia in a consumer-driven society.

5. Friendship and Community: The Epicurean Social Network in an Isolated World
   - Epicurus' emphasis on friendship as a key component of a satisfying life.
   - The challenge of building meaningful relationships in a time of social distancing and virtual interactions.
   - Suggestions for fostering community and connection in an increasingly isolated world.

6. The Gods and Mortality: Epicurean Views on Divinity and Death in a Scientific Era
   - Epicurean beliefs about the gods and the mortal soul, and their irrelevance to human affairs.
   - Discussion on how scientific advancements and AI challenge traditional views on divinity and mortality.
   - Coping with existential questions in a world where technology blurs the lines between life and death.

7. Epicureanism in Practice: The Tetrapharmakos for the Modern Individual
   - Presentation of the Tetrapharmakos as a guideline for living a happy life according to Epicureanism.
   - Application of the four-part cure to contemporary issues such as climate change, global pandemics, and economic uncertainty.
   - Encouragement for readers to embrace Epicurean wisdom in their daily lives for greater contentment and peace.

Each section will be written in an engaging and accessible style, with a focus on practical applications of Epicurean philosophy in the context of modern challenges. The chapter will avoid dry academic language and instead use relatable examples, anecdotes, and a touch of humor to keep readers engaged. The goal is to provide a fresh perspective on how ancient wisdom can inform and improve our lives in the age of AI and automation.

Active Reading Progress for EPICUREANISM [CH. 2]: 100%|██████████| 1/1 [01:12<00:00, 72.20s/it]      


Chapter Outline for MINIMALISM [CH. 3]: Living a Good Life in the Age of AI and Automation

1. Introduction to Minimalism and Its Modern Relevance
   - Brief history of minimalism as an art movement and its evolution into a lifestyle philosophy.
   - Exploration of how minimalism's principles are particularly pertinent in the era of AI and automation, where complexity and abundance often overwhelm.
   - Setting the stage for how minimalism can be a guiding philosophy in navigating the challenges of modern life, including technology saturation and the paradox of choice.

2. Minimalism in the Digital and Virtual Realm
   - Discussion on the application of minimalism to digital life, including the use of technology, social media, and the digital clutter that comes with constant connectivity.
   - Imagining a minimalist approach to virtual reality and augmented reality, focusing on the essentials and avoiding overstimulation.
   - Examples of how a minimalist mindset can enhance our interactions with AI, streamline our use of automation, and prevent digital burnout.

3. The Minimalist Approach to Work and Automation
   - How minimalism can inform our approach to the changing landscape of work, particularly with the rise of remote work and the potential for job displacement due to automation.
   - Strategies for maintaining a minimalist workspace, both physically and digitally, to enhance focus and productivity.
   - The role of minimalism in adapting to and embracing the gig economy and portfolio careers.

4. Minimalism in Personal Lifestyle Choices
   - The concept of a capsule wardrobe and its benefits in the context of consumerism and fast fashion.
   - Minimalism in home design and architecture, creating spaces that are both functional and free of excess in a world where living spaces are becoming increasingly multifunctional.
   - The minimalist kitchen: how the principles of minimalism can lead to healthier eating habits and less food waste.

5. The Social and Environmental Impact of Minimalism
   - Examining the minimalist lifestyle's potential to reduce one's carbon footprint and contribute to sustainability efforts.
   - The influence of minimalism on personal relationships and social interactions, fostering deeper connections in an age of superficial digital communication.
   - Minimalism's role in addressing the challenges of human population expansion and resource management.

6. Overcoming Challenges and Cultivating a Minimalist Mindset
   - Addressing the common obstacles faced when adopting a minimalist lifestyle, such as resistance from peers and the difficulty of decluttering.
   - Practical tips for getting started with minimalism, including how to prioritize and make intentional choices about possessions and activities.
   - The importance of maintaining a minimalist practice and how it can lead to a more intentional and fulfilling life.

7. The Future of Minimalism in an Automated World
   - Speculating on the future trajectory of minimalism as AI and automation become increasingly integrated into daily life.
   - The potential for minimalism to serve as a counterbalance to the complexities introduced by advanced technologies.
   - Closing thoughts on how minimalism can help us navigate the future with clarity, purpose, and a focus on what truly matters.

...

Active Reading Progress for NIHILISM [CH.6]: 100%|██████████| 1/1 [01:31<00:00, 91.83s/it]      


Chapter Outline for "Pragmatism and the Good Life in the Age of AI and Automation"

1. Introduction to Pragmatism: Tools for Modern Living
   - Overview of pragmatism's historical roots and key figures.
   - Explanation of pragmatism's emphasis on practical consequences and problem-solving.
   - Relevance of pragmatism in the context of AI, automation, and digital transformation.

2. Pragmatism in the Digital Workplace: Remote Work and Social Interaction
   - Analysis of how pragmatism informs remote work strategies and the cultivation of virtual communities.
   - Discussion on the pragmatist approach to maintaining work-life balance and social well-being in a remote environment.
   - Examples of pragmatic solutions to the challenges of isolation and digital communication.

3. Navigating a Jobless Future: Pragmatism and Complete Automation
   - Exploration of pragmatism's response to the potential of widespread job automation.
   - Pragmatic strategies for personal growth and adaptation in a rapidly changing job market.
   - Case studies of pragmatic education and re-skilling initiatives.

4. Pragmatism and Human Expansion: Ethics in an Automated World
   - Examination of pragmatist ethics in the face of human population expansion and environmental concerns.
   - Discussion on the pragmatic balance between technological advancement and sustainable living.
   - Insights into how pragmatism can guide ethical decision-making in an AI-dominated society.

5. Virtual and Augmented Realities: Pragmatism in Immersive Experiences
   - Analysis of pragmatism's role in shaping our engagement with virtual and augmented realities.
   - Consideration of the pragmatic implications of blending digital and physical experiences.
   - Examples of how pragmatism can help navigate the ethical and social questions raised by immersive technologies.

6. Pragmatism and the Personal Sphere: Smartphones and Video Games
   - Discussion on the pragmatist approach to the pervasive influence of smartphones and video games.
   - Pragmatic perspectives on managing digital consumption and fostering meaningful interactions.
   - Case studies of pragmatic design principles in app development and gaming.

7. Pragmatism as a Path to the Good Life: Synthesis and Future Directions
   - Summary of how pragmatism provides a framework for living well amidst technological change.
   - Reflection on the adaptability of pragmatist thought to future challenges and innovations.
   - Vision for a pragmatically-informed society that thrives in harmony with AI and automation.

Throughout the chapter, the narrative will be peppered with engaging anecdotes, thought experiments, and hypothetical scenarios that illustrate the practical application of pragmatism in the modern age. The aim is to provide readers with a dynamic and accessible understanding of how pragmatism can be a valuable guide in navigating the complexities of life in an era defined by technological disruption.

Active Reading Progress for PRAGMATISM [CH. 7]: 100%|██████████| 1/1 [01:29<00:00, 89.21s/it]       


Chapter Outline for "IDEALISM [CH. 8]"

1. Introduction to Idealism in the Age of AI and Automation
   - Briefly introduce idealism as a philosophy that equates reality with mind, spirit, or consciousness.
   - Discuss how idealism's focus on the mental and experiential aspects of existence can inform our understanding of a good life amidst the rise of AI and automation.
   - Present scenarios where idealism intersects with modern technology, such as virtual reality, augmented reality, and the potential for AI to simulate consciousness.

2. Historical Roots and Evolution of Idealism
   - Trace the origins of idealism from ancient Indian philosophy and Greek thought to its development by figures like Kant and Hegel.
   - Explore how idealism has been shaped by religious thinkers and its influence on modern religious movements.
   - Examine the criticisms and transformations of idealism in the 20th century and its resurgence in the context of quantum physics and relativity.

3. The Mind-Centric Universe: Idealism's Core Assertions
   - Delve into the metaphysical and epistemological aspects of idealism, distinguishing between subjective and objective forms.
   - Discuss the implications of a mind-centric universe for understanding reality and the potential parallels with AI's data processing.
   - Use examples from quantum physics and the "great thought" metaphor to illustrate idealism's relevance in modern scientific discourse.

4. Living the Good Life Through Idealism
   - Analyze how idealism's emphasis on consciousness and mental causation can shape one's approach to living a good life.
   - Consider the role of personal and cosmic consciousness in fostering a sense of unity and purpose in an increasingly automated world.
   - Discuss the potential for compassion, art, and aesthetic experience as idealist pathways to transcend the challenges of a technology-driven society.

5. Idealism and the Challenges of Modernity
   - Address the critiques of idealism in the face of empirical science and the materialist worldview.
   - Explore how idealism can offer a counter-narrative to the perceived threats of AI and automation, such as job displacement and loss of human connection.
   - Discuss the potential for idealism to inform ethical considerations in the development and deployment of AI technologies.

6. Idealism in Practice: Case Studies and Thought Experiments
   - Present hypothetical scenarios where idealism's principles are applied to modern dilemmas, such as the ethics of AI decision-making and the nature of virtual experiences.
   - Examine real-world examples where idealist philosophies have influenced social movements, educational approaches, and scientific theories.
   - Discuss the potential for idealism to shape future human-AI interactions and the evolution of consciousness in a digital age.

7. Conclusion: The Enduring Relevance of Idealism
   - Summarize the key points of idealism's relevance to living a good life in the context of modern challenges.
   - Highlight the potential for idealism to provide a framework for understanding and navigating the complexities of a world intertwined with AI and automation.
   - Encourage readers to consider the role of mind, spirit, and consciousness in their own lives and the collective human experience.

Active Reading Progress for IDEALISM [CH. 8]: 100%|██████████| 1/1 [02:25<00:00, 145.49s/it]      


##### Create the prompt for each chapter with a little semantic sugar around the outlines created by the document review step and submit long form job

In [5]:
chapter_ids = []

for kb, outline in outlines:
    prompt = f"""\
    You have been assigned to write a chapter in a book titled "{BOOK_TITLE}" about {kb['title']}. Please follow this outline when writing the chapter:

    {outline}

    {BOOK_OVERALL_STYLE_GUIDANCE}
    {BOOK_CHAPTER_STYLE_GUIDANCE}

    DO NOT EVER USE CLOSINGS LIKE "in conclusion", "in summary", "ultimately", etc. They are boring and you should have more pride in your output.
    """
    print('STARTING CHAPTER:', kb['title'])
    chapter_job_id = submit_long_form_job(kb, prompt)
    chapter_ids.append(chapter_job_id)
    

STARTING CHAPTER: STOICISM [CH. 1]
STARTING CHAPTER: EPICUREANISM [CH. 2]
STARTING CHAPTER: MINIMALISM [CH. 3]
STARTING CHAPTER: HEDONISM [CH. 4]
STARTING CHAPTER: EXISTENTIALISM [CH. 5]
STARTING CHAPTER: NIHILISM [CH.6]
STARTING CHAPTER: PRAGMATISM [CH. 7]
STARTING CHAPTER: IDEALISM [CH. 8]
